## **Dataset**
The Dataset is uploaded as zip and needs to be extracted.
It contains the subfolders black and white which contain moves for the respective players. The folders black and white then contain folders 1-7. Each of these folders just contains a set of positions in which the best calculated move is the column index of the parent folder.


In [2]:
# !unzip database.zip

unzip:  cannot find or open /content/database.zip, /content/database.zip.zip or /content/database.zip.ZIP.


## **Connect4Database**
The Connect4Database is an implementation of the Database object and supposed to contain the positions including the __board__, __current player__ and __target column__ for each sample.

In [1]:
from torch.utils.data import Dataset

class Connect4Dataset(Dataset):
    def __init__(self, board_data, player_data, label_data, transform=None):
        self.board_data = board_data
        self.player_data = player_data
        self.label_data = label_data
        self.transform = transform

    def __len__(self):
        return len(self.board_data)

    def __getitem__(self, index):
        board = self.board_data[index]
        player = self.player_data[index]
        label = self.label_data[index]

        # Apply the transform (if any) to the board
        if self.transform is not None:
            board = self.transform(board)

        return board, player, label

    def add(self, board, player, column):
        self.board_data.append(board)
        self.player_data.append(player)
        self.label_data.append(column)


ModuleNotFoundError: No module named 'torch'

## **Building the Datasets**
The samples are read from the file system and put into an Connect4Dataset

In [75]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

num_columns = 7

data_transforms = transforms.Compose([
    transforms.Resize((7, 6)),  # Resize the images to a consistent size (e.g., 64x64)
    transforms.Grayscale(),       # Convert images to grayscale
    transforms.ToTensor(),        # Convert images to PyTorch tensors
])


positions_white = 'connect4dataset/white'
positions_black = 'connect4dataset/black'
dataset_white = ImageFolder(positions_white, transform=data_transforms)
dataset_black = ImageFolder(positions_black, transform=data_transforms)
dataset = Connect4Dataset([], [], [])
for i in range(len(dataset_white)):
    board, label = dataset_white[i]
    dataset.add(board, 1, torch.eye(num_columns)[label - 1])
for i in range(len(dataset_black)):
    board, label = dataset_black[i]
    dataset.add(board, 0, torch.eye(num_columns)[label - 1])

train_size = int(0.6 * len(dataset))  # 60% for training
test_size = int(0.2 * len(dataset))  # 20% for testing
val_size = len(dataset) - train_size - test_size  # 20% for validation
train_dataset, test_dataset, validation_dataset = torch.utils.data.random_split(dataset, [train_size, test_size, val_size])

# Create data loaders for training and testing
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## **Model**
As model we use a CNN. It operates on the image of the board and stacks mulitple convolutional layers together to enable Pattern recognition.
It is then followed by a four layer dense neural network. The first vector of the dense neural network also receives the current player as second input. The produced output vector has length 7. It contains the estimated probabilites of putting the next token in the respective column being the best move.

In [84]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

class Bot(torch.nn.Module):

    def __init__(self):
        super(Bot, self).__init__()

        self.conv1 = nn.Conv2d(1, 55, (2, 2))
        #self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(55, 1000, (2, 2))
        #self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(1000, 1000, (2, 2))
        #self.pool3 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(1000, 1000, (2, 2))
        #self.pool4 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(3 * 2 * 1000 + 1, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 7)
        print(self)

    def forward(self, board, current_player):
        x = F.relu(self.conv1(board))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = torch.flatten(x, 1)
        current_player = current_player.unsqueeze(1)
        x = torch.cat((x, current_player), dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

bot = Bot()

Bot(
  (conv1): Conv2d(1, 55, kernel_size=(2, 2), stride=(1, 1))
  (conv2): Conv2d(55, 1000, kernel_size=(2, 2), stride=(1, 1))
  (conv3): Conv2d(1000, 1000, kernel_size=(2, 2), stride=(1, 1))
  (conv4): Conv2d(1000, 1000, kernel_size=(2, 2), stride=(1, 1))
  (fc1): Linear(in_features=6001, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=7, bias=True)
)


## **Training the model**
As loss function we use CrossEntropy and as Optimizer SGD.

In [85]:
binary_cross_entropy = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(bot.parameters(), lr=0.01)

num_epochs = 10  # Number of training epochs

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (board, current_player, label) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = bot(board, current_player)
        loss = binary_cross_entropy(outputs, label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # Print the average loss every 1000 mini-batches
        if i % 1000 == 999:
            print(f"[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 1000:.3f}")
            running_loss = 0.0

print("Training completed.")

Training completed.


## **Testing the model**
We give the model the test subset and calculate the accuracy.


In [86]:
bot.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for board, current_player, label in test_loader:
        outputs = bot(board, current_player)
        predicted_indices = torch.argmax(outputs, dim=1)
        labeled_indices = torch.argmax(label, dim=1)
        total += label.size(0)
        correct += (predicted_indices == labeled_indices).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on the test set: {accuracy:.2f}%")

bot.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for board, current_player, label in train_loader:
        outputs = bot(board, current_player)
        predicted_indices = torch.argmax(outputs, dim=1)
        labeled_indices = torch.argmax(label, dim=1)
        total += label.size(0)
        correct += (predicted_indices == labeled_indices).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on the training set: {accuracy:.2f}%")

Accuracy on the test set: 13.33%
Accuracy on the train set: 13.72%
